In [70]:
from gemlib.database.utils import load_data_azure
import pandas as pd
import math 
import numpy as np
# from rich.jupyter import display
from scipy.stats import norm
import warnings
from IPython.display import display

warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None

In [67]:
from datetime import datetime

In [68]:
dt = datetime.now()
data_calculo = int(str(dt.year) + str(dt.month).zfill(2) + str(dt.day).zfill(2))

In [71]:
ini = int(str(dt.year) + str(dt.month).zfill(2)) + 1
fim = 202512

qr_fml_vol = f"""
    SELECT
        ano_aplic * 100 + mes_aplic AS mes_aplic,
        id_volat_serie AS id,
        SUM(valor) AS S
    FROM udm_formulador_volat
    WHERE
        id_formula = 484
        AND id_volat_serie <= 90
        AND data_cotacao <= CAST('{data_calculo}' AS DATE)
        AND ano_aplic * 100 + mes_aplic BETWEEN {ini} AND {fim}
    GROUP BY
        id_volat_serie,
        mes_aplic,
        ano_aplic
    ORDER BY
        ano_aplic ASC,
        mes_aplic ASC,
        id_volat_serie DESC
"""

bd_S = load_data_azure(qr_fml_vol)

display(bd_S)


,mes_aplic,id,S
0,202501.0,90,546.09856
1,202501.0,89,534.80404
2,202501.0,88,527.92828
3,202501.0,87,528.06973
4,202501.0,86,530.24629
...,...,...,...
1075,202512.0,5,585.42622
1076,202512.0,4,588.21272
1077,202512.0,3,589.32435
1078,202512.0,2,631.76711


In [13]:
bd_S = bd_S.rename(columns={'s':'S'})

bd_S["mês"] = bd_S["mes_aplic"] % 100
bd_S["ano"] = bd_S["mes_aplic"] // 100

dtypes = {
    "mes_aplic": "int64",
    "id": "int64",
    "S": "float64",
    "mês": "int64",
    "ano": "int64"
}

bd_S = bd_S.astype(dtypes)

display(bd_S)

,mes_aplic,id,S,mês,ano
0,202501,90,546.58656,1,2025
1,202501,89,546.09856,1,2025
2,202501,88,534.80404,1,2025
3,202501,87,527.92828,1,2025
4,202501,86,528.06973,1,2025
...,...,...,...,...,...
265,202503,5,553.29185,3,2025
266,202503,4,564.37668,3,2025
267,202503,3,583.78652,3,2025
268,202503,2,586.99188,3,2025


In [38]:
## Forma_Horaria:
    # 1 - Mensal ; 2 - Diária ; 3 - Horária 


##SPOT --> Romania
    # id_spot = 128
    # id_tipo = 1 (Diario)
    # id_valor = 5 (CLOSE)
# Vol_Form(2019,2022,2023,3)
# def Vol_Form(Ini, Fim, Hor, f):


hor = str(2023) + "1231"

ELECTR_HOR1 = f"""
    SELECT
        '_MM_' + RIGHT('0' + CAST(MONTH(CONVERT(DATE, CONVERT(VARCHAR(8), dia_aplic))) AS NVARCHAR), 2) +
        '_TD_' + CAST(gem.get_week_day_sunday(CONVERT(DATE, CONVERT(VARCHAR(8), dia_aplic))) AS NVARCHAR) +
        '_HH_' + RIGHT('0' + CAST(hora_aplic AS NVARCHAR(2)), 2) AS chave,
        dia_aplic,
        media_mes,
        valor,
        horas_dia
    FROM (
        SELECT
            dia_aplic,
            hora_aplic,
            valor,
            media_mes,
            COUNT(*) AS horas_dia,
            DENSE_RANK() OVER (
                PARTITION BY MONTH(CONVERT(DATE, CONVERT(VARCHAR(8), dia_aplic)))
                ORDER BY YEAR(CONVERT(DATE, CONVERT(VARCHAR(8), dia_aplic))) DESC
            ) AS melhorAno
        FROM (
            SELECT
                dia_aplic,
                hora_aplic,
                valor,
                FLOOR(dia_aplic / 100) AS mes_aplic,
                COUNT(*) OVER (PARTITION BY FLOOR(dia_aplic / 100)) AS dias_existentes,
                DAY(EOMONTH(CONVERT(DATE, CONVERT(VARCHAR(8), dia_aplic)))) AS dias_esperados,
                AVG(valor) OVER (PARTITION BY FLOOR(dia_aplic / 100)) AS media_mes
            FROM
                udm_dmo_spot_valor
            WHERE
                id_spot = 128
                AND id_tipo_spot = 1
                AND id_tipo_valor = 5
                AND referencia = 1
                AND MONTH(CONVERT(DATE, CONVERT(VARCHAR(8), dia_aplic))) IN (1,2,3,4,5,6,7,8,9,10,11,12)
                AND dia_aplic <= {hor}
        )  base_data
        WHERE dias_existentes = dias_esperados
        GROUP BY
            dia_aplic,
            hora_aplic,
            valor,
            FLOOR(dia_aplic / 100),
            media_mes
    ) ranked_data
    WHERE melhorAno = 1
    ORDER BY dia_aplic, hora_aplic;

"""

hora1 = load_data_azure(ELECTR_HOR1)

dtype = {
    "chave": str,
    "dia_aplic": "int64",
    "media_mes": "float64",
    "valor": "float64",
    "horas_dia": "int64"
}

hora1 = hora1.astype(dtype)
display(hora1)

,chave,dia_aplic,media_mes,valor,horas_dia
0,_MM_01_TD_1_HH_-1,20230101,670.714516,88.39,1
1,_MM_01_TD_2_HH_-1,20230102,670.714516,617.55,1
2,_MM_01_TD_3_HH_-1,20230103,670.714516,739.12,1
3,_MM_01_TD_4_HH_-1,20230104,670.714516,930.73,1
4,_MM_01_TD_5_HH_-1,20230105,670.714516,682.86,1
...,...,...,...,...,...
360,_MM_12_TD_4_HH_-1,20231227,402.346129,276.99,1
361,_MM_12_TD_5_HH_-1,20231228,402.346129,211.06,1
362,_MM_12_TD_6_HH_-1,20231229,402.346129,112.10,1
363,_MM_12_TD_7_HH_-1,20231230,402.346129,180.94,1


In [56]:
# Vol_Form(2019,2022,2023,3)
# def Vol_Form(Ini, Fim, Hor, f):

ini = str(2019) + "0101"
fim = str(2022) + "1231"

ELECTR_HOR2 = f"""
SELECT
    '_MM_' + RIGHT('0' + CAST(mes AS VARCHAR(2)), 2) +
    '_TD_' + CAST(tipo_dia_aplic AS VARCHAR) +
    '_HH_' + RIGHT('0' + CAST(hora_aplic AS VARCHAR(2)), 2) AS chave,
    media_mes,
    coef_hora,
    coef_dia
FROM (
    SELECT DISTINCT
        mes,
        tipo_dia_aplic,
        hora_aplic,
        media_mes,
        AVG(CASE WHEN media_dia = 0 THEN 0 ELSE CAST(valor AS FLOAT) / media_dia END)
        OVER (PARTITION BY mes, tipo_dia_aplic, hora_aplic) AS coef_hora,
        CASE WHEN mes = 0 THEN 0 ELSE media_mes_tipo_dia / media_mes END AS coef_dia
    FROM (
        SELECT
            tab1.*,
            AVG(CAST(valor AS FLOAT)) OVER (PARTITION BY mes) AS media_mes,
            AVG(CAST(valor AS FLOAT)) OVER (PARTITION BY mes, tipo_dia_aplic) AS media_mes_tipo_dia,
            AVG(CAST(valor AS FLOAT)) OVER (PARTITION BY dia_aplic) AS media_dia
        FROM (
            SELECT
                (FLOOR(dia_aplic / 100) % 100) AS mes,
                FLOOR(dia_aplic / 100) AS mes_aplic,
                dia_aplic,
                gem.get_week_day_sunday(CAST(dia_aplic AS VARCHAR(8))) AS tipo_dia_aplic,
                hora_aplic,
                valor
            FROM udm_dmo_spot_valor
            WHERE
                id_spot = 128
                AND id_tipo_spot = 1
                AND id_tipo_valor = 5
                AND referencia = 1
                AND (FLOOR(dia_aplic / 100) % 100) IN (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12)
                AND dia_aplic BETWEEN {ini} AND {fim}
        ) tab1
    ) tab2
) subquery
ORDER BY
    mes,
    tipo_dia_aplic,
    hora_aplic;
"""

hora2 = load_data_azure(ELECTR_HOR2)

dtype = {
    "chave": str,
    "media_mes": "float64",
    "coef_hora": "float64",
    "coef_dia": "float64"
}

hora2 = hora2.astype(dtype)

display(hora2)


,chave,media_mes,coef_hora,coef_dia
0,_MM_01_TD_1_HH_-1,494.803804,1.0,0.855838
1,_MM_01_TD_2_HH_-1,494.803804,1.0,1.062959
2,_MM_01_TD_3_HH_-1,494.803804,1.0,1.055410
3,_MM_01_TD_4_HH_-1,494.803804,1.0,1.063576
4,_MM_01_TD_5_HH_-1,494.803804,1.0,1.051655
...,...,...,...,...
79,_MM_12_TD_3_HH_-1,878.655457,1.0,1.036476
80,_MM_12_TD_4_HH_-1,878.655457,1.0,1.125014
81,_MM_12_TD_5_HH_-1,878.655457,1.0,1.129750
82,_MM_12_TD_6_HH_-1,878.655457,1.0,1.014806


In [43]:

df_forma = pd.merge(hora1, hora2, how="left", on="chave")
df_forma = df_forma.drop("media_mes_y", axis=1)
df_forma["Estimativa_Hora"] = df_forma["media_mes_x"] * df_forma["coef_hora"] * df_forma["coef_dia"]

display(df_forma)

,chave,dia_aplic,media_mes_x,valor,horas_dia,coef_hora,coef_dia,Estimativa_Hora
0,_MM_01_TD_1_HH_-1,20230101,670.714516,88.39,1,1.0,0.855838,574.023301
1,_MM_01_TD_2_HH_-1,20230102,670.714516,617.55,1,1.0,1.062959,712.942027
2,_MM_01_TD_3_HH_-1,20230103,670.714516,739.12,1,1.0,1.055410,707.878740
3,_MM_01_TD_4_HH_-1,20230104,670.714516,930.73,1,1.0,1.063576,713.356154
4,_MM_01_TD_5_HH_-1,20230105,670.714516,682.86,1,1.0,1.051655,705.360521
...,...,...,...,...,...,...,...,...
360,_MM_12_TD_4_HH_-1,20231227,402.346129,276.99,1,1.0,1.125014,452.644843
361,_MM_12_TD_5_HH_-1,20231228,402.346129,211.06,1,1.0,1.129750,454.550385
362,_MM_12_TD_6_HH_-1,20231229,402.346129,112.10,1,1.0,1.014806,408.303344
363,_MM_12_TD_7_HH_-1,20231230,402.346129,180.94,1,1.0,0.848161,341.254408


In [46]:
f=3

if f==3:
    df_forma["Erro"] = df_forma.apply(lambda x: min((x["Estimativa_Hora"] / x["valor"]), 2), axis=1)
    volat_formula = df_forma["Erro"].std()
elif f==2:
    x1 = df_forma.groupby("dia_aplic").apply(lambda x: min((x["Estimativa_Hora"].mean()) / (x["valor"].mean()),2)).reset_index()
    x1.rename(columns={0 :'Erro'}, inplace=True )
    volat_formula = x1["Erro"].std()
else:
    volat_formula = 0

In [44]:
display(df_forma)

,chave,dia_aplic,media_mes_x,valor,horas_dia,coef_hora,coef_dia,Estimativa_Hora
0,_MM_01_TD_1_HH_-1,20230101,670.714516,88.39,1,1.0,0.855838,574.023301
1,_MM_01_TD_2_HH_-1,20230102,670.714516,617.55,1,1.0,1.062959,712.942027
2,_MM_01_TD_3_HH_-1,20230103,670.714516,739.12,1,1.0,1.055410,707.878740
3,_MM_01_TD_4_HH_-1,20230104,670.714516,930.73,1,1.0,1.063576,713.356154
4,_MM_01_TD_5_HH_-1,20230105,670.714516,682.86,1,1.0,1.051655,705.360521
...,...,...,...,...,...,...,...,...
360,_MM_12_TD_4_HH_-1,20231227,402.346129,276.99,1,1.0,1.125014,452.644843
361,_MM_12_TD_5_HH_-1,20231228,402.346129,211.06,1,1.0,1.129750,454.550385
362,_MM_12_TD_6_HH_-1,20231229,402.346129,112.10,1,1.0,1.014806,408.303344
363,_MM_12_TD_7_HH_-1,20231230,402.346129,180.94,1,1.0,0.848161,341.254408


In [57]:
display(volat_formula)

0.300627258305945

In [48]:
def params(df_S, form_vol):

    S_volat = df_S.copy()
    #MM(5)
    S_volat["MM_S"] = S_volat.groupby("mes_aplic")["S"].rolling(window=5).mean().reset_index(0, drop=True)
    S_volat = S_volat.dropna()

    #Log-returns
    Log_S = S_volat.groupby("mes_aplic").apply(lambda x: np.log(x["MM_S"] / x["MM_S"].shift(1)))
    S_volat["Log_S"] = Log_S.reset_index(level=0, drop=True)
    S_volat = S_volat.dropna()

    #Outliers
    Q1_S = S_volat.groupby("ano")["Log_S"].quantile(0.25)
    Q3_S = S_volat.groupby("ano")["Log_S"].quantile(0.75)
    IQ_S1 = (Q1_S - (3.4*(abs(Q3_S - Q1_S)))).to_dict()
    IQ_S2 = (Q3_S + (3.4*(abs(Q3_S + Q1_S)))).to_dict()

    S_volat["IQ_S1"] = S_volat["ano"].map(IQ_S1)
    S_volat["IQ_S2"] = S_volat["ano"].map(IQ_S2)

    S_volat = S_volat.loc[(S_volat["Log_S"] > S_volat["IQ_S1"]) & (S_volat["Log_S"] < S_volat["IQ_S2"])]

    S_volat = (S_volat.groupby("mes_aplic")["Log_S"].std())*math.sqrt(253)
    S_volat.to_dict()

    bd = df_S.loc[bd_S['id'] == 1]

    bd["today"] = data_calculo
    bd["today"] = bd["today"].astype("string")
    bd["today"] = bd["today"].apply(pd.Timestamp)

    bd["Expiry"] = pd.to_datetime(dict(year=bd.ano , month=bd.mês, day=15), format="%Y%m%d")
    bd["Expiry"] = bd["Expiry"].apply(pd.Timestamp)

    bd["days"] = bd.apply(lambda x: len(pd.bdate_range(x.today, x.Expiry)), axis=1)

    bd["t"] = bd["days"] / 253
    t = bd.groupby("mes_aplic")["t"].mean()

    df_S = bd.copy()
    df_S["Sigma"] = df_S["mes_aplic"].map(S_volat)
    df_S["t"] = df_S["mes_aplic"].map(t)
    df_S["Volat_Forma"] = form_vol

    x = ['id', 'mês', 'ano', 'days', 'today', 'Expiry']
    df_S = df_S.drop(x, axis=1)
           
    return df_S


In [50]:
df = params(bd_S, volat_formula)

display(df)

,mes_aplic,S,t,Sigma,Volat_Forma
89,202501,700.99681,0.071146,0.189376,0.300627
179,202502,647.40296,0.158103,0.182758,0.300627
269,202503,617.05623,0.237154,0.209595,0.300627


#### Black Scholes

In [ ]:
#K = 450 

In [51]:
def black_scholes (S, K, Sigma_Spread, Volat_Forma, T, q_K, q_S, r):
    d1 = (math.log(S/K)+ ((q_K - q_S) + ((Sigma_Spread**2/2)*T + (Volat_Forma**2)/2)) + (r**2)/2) / (Sigma_Spread*math.sqrt(T) + r + Volat_Forma)
    d2 = d1 - Sigma_Spread*math.sqrt(T) - r - Volat_Forma
    call = S *norm.cdf(d1) - K *norm.cdf(d2)
    put = -S *norm.cdf(-d1) + K *norm.cdf(-d2)  
    delta_1 = norm.cdf(d1)
    delta_2 = norm.cdf(d2)
    return d1, d2, call, put, delta_1, delta_2

for i, r in df.iterrows():
    d1, d2, call, put, delta_1_call, delta_2_call = black_scholes(r["S"], 450, r["Sigma"], r["Volat_Forma"], r["t"], 0, 0, 0)
    df.loc[i, "d1"] = d1
    df.loc[i, "d2"] = d2
    df.loc[i, "call"]= call  
    df.loc[i, "put"] = put
    df.loc[i, "delta_1"] = delta_1_call
    df.loc[i, "delta_2"]= delta_2_call

display(df)


,mes_aplic,S,t,Sigma,Volat_Forma,d1,d2,call,put,delta_1,delta_2
89,202501,700.99681,0.071146,0.189376,0.300627,1.394657,1.043517,260.583260,9.586450,0.918440,0.851646
179,202502,647.40296,0.158103,0.182758,0.300627,1.102478,0.729182,214.746638,17.343678,0.864873,0.767055
269,202503,617.05623,0.237154,0.209595,0.300627,0.909145,0.506448,192.798103,25.741873,0.818363,0.693729


In [59]:
df["Deltas_Rom"] = 1 - df["delta_1"]

display(df)

,mes_aplic,S,t,Sigma,Volat_Forma,d1,d2,call,put,delta_1,delta_2,Deltas_Rom,date
89,202501,700.99681,0.071146,0.189376,0.300627,1.394657,1.043517,260.583260,9.586450,0.918440,0.851646,0.081560,2025-01-01
179,202502,647.40296,0.158103,0.182758,0.300627,1.102478,0.729182,214.746638,17.343678,0.864873,0.767055,0.135127,2025-02-01
269,202503,617.05623,0.237154,0.209595,0.300627,0.909145,0.506448,192.798103,25.741873,0.818363,0.693729,0.181637,2025-03-01


In [53]:
def convert_date(date):
    return pd.to_datetime(date, format='%Y%m').strftime('%Y-%m-01')

In [64]:
df["date"] = df["mes_aplic"].apply(convert_date)

In [65]:
df["Deltas_Rom"].values

array([0.08155962, 0.13512704, 0.18163672])

In [30]:
# ## XML
#
# import datetime
# import time
#
# now = datetime.datetime.now()
# todays_Date = datetime.date.fromtimestamp(time.time())
# date = todays_Date.isoformat()
# now_str = now.strftime('%Y%m%d%H%M%S')
# now = now.isoformat(timespec='milliseconds')
# date_iso = datetime.datetime.now()
# #date_iso = date_iso.replace(hour=0, minute=0, second=0, microsecond=0)
# date_iso = date_iso.isoformat(timespec='milliseconds')
# xml_path = ('PTD_DMO_' + now_str +'_PREVISOES_ROM_DELTAS.xml')


In [31]:
# pad = '  '
#
# xml_info = ['<Previsao>\n',
#               pad + '<Info>\n',
#                   pad*2 + '<Descricao>UNGE_Deltas_Romenia</Descricao>\n',
#                   pad*2 +'<Provedor>GEM</Provedor>\n',
#                   pad*2 + '<UnidadeTemporal>Mensal</UnidadeTemporal>\n',
#                   pad*2 +'<LOG_TimeSTAMP>' + date_iso + '</LOG_TimeSTAMP>\n',
#                   pad*2 +'<Versao>' + now_str[8:10] + '</Versao>\n',
#                   pad*2 +'<TimeZone>UTC</TimeZone>\n',
#               pad +'</Info>']
#
# xml_info = ''.join(xml_info)


In [32]:
# xml_rom = []
#
# for ind in df.index:
#     delt = [pad + '<Linha>\n',
#                      pad*2 + '<MODELO>Deltas_Exposicao</MODELO>\n',
#                      pad*2 + '<DATA_APLIC>' + str(df.loc[ind,'date']) + '</DATA_APLIC>\n' ,
#                      pad*2 + '<PERCENTIL>50</PERCENTIL>\n',
#                      pad*2 + '<HORA>-1</HORA>\n',
#                      pad*2 + '<VALOR>' + str(df.loc[ind,'Deltas_Rom']) + '</VALOR>\n' ,
#                      pad*2 + '<UnidadePrevisao>Percentagem</UnidadePrevisao>\n',
#                      pad*2 + '<ZONA>Roménia</ZONA>\n',
#             pad + '</Linha>']
#
#     xml = ''.join(delt)
#
#     xml_rom.append(xml)
#
# xml_euro = ''.join(xml_rom)
#
# xml_all =  xml_info + xml_euro
#
# xml_all = xml_all + '\n' + '</Previsao>'

In [33]:
# import xml.etree.ElementTree as ET
# from lxml import etree
#
# parser = etree.XMLParser(recover=True)
#
# root = etree.fromstring(xml_all, parser=parser)
# my_tree = etree.ElementTree(root)
#
#
# my_tree.write(xml_path)